In [ ]:
import numpy as np
import matplotlib.pyplot as plt

with np.load('prediction-challenge-02-data.npz') as fh:
    x_train = fh['x_train']
    y_train = fh['y_train']
    x_test = fh['x_test']

# TRAINING DATA: INPUT (x) AND OUTPUT (y)
# 1. INDEX: IMAGE SERIAL NUMBER (6000)
# 2/3. INDEX: PIXEL VALUE (32 x 32)
# 4. INDEX: COLOR CHANNELS (3)
print(x_train.shape, x_train.dtype)
print(y_train.shape, y_train.dtype)

# TEST DATA: INPUT (x) ONLY
print(x_test.shape, x_test.dtype)

# TRAIN MODEL ON x_train, y_train

# Data Stuff

In [ ]:
from matplotlib import pyplot as plt
i = 4
plt.imshow(x_train[i])
y_train[i]

In [ ]:
def swap(x):
    return x.swapaxes(-1,-2).swapaxes(-2,-3)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_training, x_validation, y_training, y_validation = train_test_split(
    swap(x_train), y_train
)

In [ ]:
y_training.shape

In [ ]:
import torch

# Network

In [ ]:
class DenseNet(torch.nn.Sequential):
    def __init__(self, *sizes, activation=torch.nn.ReLU()):
        layers = self._make_layers(*sizes, activation=activation)
        super().__init__(*layers)
    
    @staticmethod
    def _make_layers(*sizes, activation):
        layers = []
        for i in range(len(sizes)-1):
            layers.append(torch.nn.Linear(sizes[i], sizes[i+1]))
            if i < len(sizes)-2:
                layers.append(activation)
        return layers
    
DenseNet(128, 2, 12, 24)

In [ ]:
class RegularizedConvBlock(torch.nn.Sequential):
    def __init__(
        self, 
        in_features, 
        out_features, 
        kernel_size=2,
        dropout=0.2,
        activation=torch.nn.ReLU()
    ):
        layers = [
            torch.nn.Conv2d(in_features, out_features, kernel_size),
            activation,
            torch.nn.BatchNorm2d(out_features),
            torch.nn.MaxPool2d(kernel_size),
            torch.nn.Dropout(dropout)
        ]
        super().__init__(*layers)
    
    def forward(self, x):
        dx = super().forward(x)
        return dx
        
        
RegularizedConvBlock(3,3)(torch.tensor(x_training[:10])).shape

In [ ]:
class Classifier(torch.nn.Sequential):
    def __init__(self, *hidden, dropout=0.5, activation=torch.nn.ReLU()):
        layers = []
        layers.append(RegularizedConvBlock(3,64))
        layers.append(RegularizedConvBlock(64,16))
        layers.append(RegularizedConvBlock(16,3))
        layers.append(torch.nn.Flatten())
        layers.append(DenseNet(27, *hidden, 3))
        layers.append(torch.nn.LogSoftmax(dim=1))
        super().__init__(*layers)
        
Classifier(128,128)(torch.tensor(x_training[:10])).shape

In [ ]:
class NaiveClassifier(torch.nn.Sequential):
    def __init__(self, *hidden):
        layers = []
        layers.append(torch.nn.Flatten())
        layers.append(DenseNet(3*32*32, *hidden, 3))
        layers.append(torch.nn.LogSoftmax(dim=1))
        super().__init__(*layers)

# Training

In [ ]:
device = torch.device("cuda:0")

In [ ]:
classifier = Classifier(512, 64, dropout=0.5)
#classifier = NaiveClassifier(2048,1024,256,16)
classifier.to(device)
#train_loader.to(device)
#validation_loader.to(device)

In [ ]:
def accuracy(x,y):
    return (torch.argmax(x, dim=-1) == y).sum()/len(y)

In [ ]:
optim = torch.optim.Adam(classifier.parameters(), lr=5e-4)
loss = torch.nn.NLLLoss()

In [ ]:
n_epoch = 200

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
to_tensor = lambda x: torch.tensor(x, dtype=torch.float32, device=device)

In [ ]:
for epoch in range(n_epoch):

    train_loader = DataLoader(
        TensorDataset(
            to_tensor(x_training), 
            to_tensor(y_training).to(dtype=int)
        ), batch_size=64, shuffle=True
    )
    validation_loader = DataLoader(
        TensorDataset(
            to_tensor(x_validation), 
            to_tensor(y_validation).to(dtype=int)
        ), 
        batch_size=64, shuffle=False
    )

    all_nlls = []
    all_accuracies = []
    for x,y in train_loader:
        optim.zero_grad()
        prediction = classifier(x)
        nll = loss(prediction, y)
        nll.backward()
        optim.step()
        print(accuracy(prediction,y).item(), end="\r")
        all_nlls.append(nll.item())
        all_accuracies.append(accuracy(prediction,y).item())
    train_string = (
        "Train. Accuracy  {:.2f}   Error {:.4f}".format(
            np.mean(all_accuracies), np.mean(all_nlls)
        )   
    )
    
    # validation
    with torch.no_grad():
        classifier.train(False)
        all_nlls = []
        all_accuracies = []
        for x,y in validation_loader:
            prediction = classifier(x)
            nll = loss(prediction, y)
            all_nlls.append(nll.item())
            all_accuracies.append(accuracy(prediction,y).item())
        test_string = (
            "Val. Accuracy  {:.2f}   Error {:.4f}".format(
                np.mean(all_accuracies), np.mean(all_nlls)
            )   
        )
        classifier.train(True)
    print(f"{epoch+1:2d}/{n_epoch} | {train_string}  |  " + test_string)
        

In [ ]:
prediction = classifier(to_tensor(swap(x_test))).argmax(dim=-1).detach().cpu().numpy()

In [ ]:
prediction

In [ ]:
# 0:cat, 1:dog, 2:frog

In [ ]:
i = 12
plt.imshow(x_test[i])
prediction[i]

In [ ]:
# MAKE SURE THAT YOU HAVE THE RIGHT FORMAT
assert prediction.ndim == 1
assert prediction.shape[0] == 300

# AND SAVE EXACTLY AS SHOWN BELOW
np.save('prediction.npy', prediction.astype(int))

# MAKE SURE THAT THE FILE HAS THE CORRECT FORMAT
def validate_prediction_format():
    loaded = np.load('prediction.npy')
    assert loaded.shape == (300, )
    assert loaded.dtype == int
    assert (loaded <= 2).all()
    assert (loaded >= 0).all()
validate_prediction_format()